In [1]:
import pandas as pd
from pathlib import Path
import numpy as np
from time import time
import sys
from __future__ import print_function
from datetime import datetime
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw, ImageFont
import math




from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import LSTM, Embedding, TimeDistributed, Dense, RepeatVector, Concatenate,\
                         Activation, Flatten, Reshape, concatenate, Dropout, BatchNormalization
from keras.optimizers import Adam, RMSprop
from keras.layers.wrappers import Bidirectional
from keras.layers.merge import add
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras import Input, layers
from keras import optimizers
from keras.applications.inception_v3 import preprocess_input
from keras.utils import to_categorical
from keras.initializers import Constant
from keras.utils.data_utils import get_file
from keras.callbacks import LambdaCallback
from keras.models import load_model
from keras.utils import plot_model
from tensorflow.python import keras as K



/home/jeremi/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# 0. Importer les images et leur captions manuellement
Output : 
1 liste d'images (format: liste de liste de pixels)
1 liste de captions (format: liste de phrase)


## Importer les données et créer un dictionnaire des captions liées au nom de fichier de l'image

In [2]:
data = pd.read_csv('data/liste_memes.csv', sep=',')
data.columns = ['num', 'image','RGB', 'caption']

captions = data['caption'].tolist()
images = data['image'].tolist()

caption_img = dict(zip(captions, images))

In [3]:
# sanity check : est-ce que la caption a un fichier lié?
caption_img['Quand une fille se maquille  ']

'quebecois1038.jpeg'

In [4]:
#Certains memes sont dans le dictionnaire mais on n'a pas les fichiers. Effacer ces entrées du dict

memes_path = 'data/all_memes/'

for key, value in list(caption_img.items()):
  my_file = Path(memes_path + value)
  if not my_file.exists():
    del caption_img[key]


In [5]:
# images & captions, prêtes à être entraînées
caption_list = list(caption_img.keys())
img_list = list(caption_img.values())

print('Il y a ' +str(len(img_list)) + ' images prêtes')
print('Il y a ' +str(len(caption_list)) + ' captions prêtes') 

Il y a 2841 images prêtes
Il y a 2841 captions prêtes


In [6]:
# regarder quelques captions
caption_list[:10]

['Voisins qui habitent en haut de chez toi: STARTER PACK',
 '2019 est presque terminé et tout ce que je peux dire c‘est:',
 'Moi qui check mon lit une dernière fois le matin avant de quitter pour la job',
 '*Arrive à la job*  9:00am: Hehe ça va être tellement relaxe aujourd’hui  9:04am:',
 'Moi: J’dois partir a 8:30pm. J’devrais commencer a me préparer à 7:30pm.  Moi a 8:24pm:  a) e  rer aE',
 'Quand tu lui a nommé tous les restos du Québec et elle ne sait toujours pas ce qu’elle veut manger...',
 'Quand tu fais juste TROP penser sur TOUT alors tu te fais du mal à toimême',
 'Quand un beau gars te cruise au bar et te dit qu‘il t‘offre un verre:',
 'Quand ton amoureux est plus vieux que toi de 2 ans ou plus:',
 'Ma blonde: Le costume de chien parfait n’existe pas.']

# 1. Tokenize les captions
Output : liste de captions. Chaque mot est remplacé par un index en integer 

In [7]:
# ajouter le SOS et EOS à chaque caption

mark_start = 'ssss '
mark_end = ' eeee'

def mark_captions(captions_list):
    captions_marked = [mark_start + str(caption) + mark_end
                        for caption in captions_list]
                        
    
    return captions_marked

captions_train_marked = mark_captions(caption_list)

#sanity check : regarder le résultat
captions_train_marked[103]

'ssss Je suis: Un gars Une fille ” Etudiant(e) Je recherche: Une blonde  Un chum   La fin de cette osti d’session a marde eeee'

In [8]:
#enlever manuellement certains caractères qui ne seront pas enlevés par keras
exclude = ['«', '»','"', '”']

def remove_exclude(caption):
    return ''.join(ch for ch in caption if ch not in exclude)

# enlever les guillemets
captions_train_marked2 = captions_train_marked
captions_train_marked = []

for caption in captions_train_marked2:
  captions_train_marked.append(remove_exclude(caption))

#sanity check : regarder le résultat
captions_train_marked[103]

'ssss Je suis: Un gars Une fille  Etudiant(e) Je recherche: Une blonde  Un chum   La fin de cette osti d’session a marde eeee'

## Tokenize avec un dictionnaire

In [9]:
# le nombre de mots max dans notre dictionnaire
MAX_NB_WORDS = 6000

In [10]:
# on veut connaître la longueur de la caption la plus longue
MAX_SEQUENCE_LENGTH = 0
for caption in captions_train_marked:
  if len(caption.split()) > MAX_SEQUENCE_LENGTH:
    MAX_SEQUENCE_LENGTH = len(caption.split())
  
print(MAX_SEQUENCE_LENGTH)

105


In [11]:
# transforme chaque caption en une séquence de nombre entier, qui représente le mot
tokenizer = Tokenizer(nb_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n«»', oov_token=True) #on ajoute les «» dans les filters, et on met un token pour les mots hors du 7000
tokenizer.fit_on_texts(captions_train_marked)
sequences = tokenizer.texts_to_sequences(captions_train_marked)


/home/jeremi/anaconda3/lib/python3.6/site-packages/keras_preprocessing/text.py:178: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `Tokenizer` '


In [12]:
print(sequences[1000])
print(captions_train_marked[1000])

[2, 34, 38, 211, 18, 44, 1820, 792, 40, 11, 31, 2544, 50, 24, 2545, 4, 33, 64, 3]
ssss On ne peut pas se marier sauf si tu me laisses faire des conneries de ce genre: eeee


In [13]:
#crée un dictionnaire avec chaque mot et leur index
word_index = tokenizer.word_index
print('Il y a %s tokens uniques.' % len(word_index))

#par convention keras, on ajoute +1
num_words_k = len(word_index)+1

# créer un dictionnaire avec chaque index et leur mot
index_word = dict(map(reversed, tokenizer.word_index.items()))

Il y a 7366 tokens uniques.


In [14]:
#sanity check : vérifier l'index du mot brave, et ce que l'index retourne
print(word_index['porte'])
print(index_word[236])

236
porte


In [15]:
# le mot FBI est hors des premiers 6000. Qcq Keras en fait? Il met index 1
index_phrase_unknown = caption_list.index('J‘ai jamais compris pourquoi c‘est surtout des hommes qui travaillent pour le FBI parce que les capacités que détiennent une femme pour démasquer des trucs sont incomparables  ')
print(captions_train_marked[index_phrase_unknown])
print(sequences[index_phrase_unknown])

ssss J‘ai jamais compris pourquoi c‘est surtout des hommes qui travaillent pour le FBI parce que les capacités que détiennent une femme pour démasquer des trucs sont incomparables   eeee
[2, 47, 100, 1145, 118, 57, 2094, 24, 1469, 15, 1616, 19, 12, 1, 362, 6, 17, 1, 6, 1, 16, 873, 19, 1, 24, 184, 97, 1, 3]


In [16]:
#le mot avec l'index 1 est un token True qui dit 'hors du top 6000'
print(index_word[1])

True


In [17]:
# prendre l'index des tokens start et end
start_tok = word_index['ssss']
end_tok = word_index['eeee']
print(start_tok, ', ', end_tok)

2 ,  3


In [18]:
#sanity check : regarder c'est quoi l'index max. ça devrait être 1 de moins que le nb de mots max (donc 5999 ici)
max_val = 0
for caption in sequences:
  max_cap = max(caption)
  if max_cap >= max_val:
    max_val = max_cap

print(max_val)

5999


# 2. Bâtir l'encodeur

In [19]:
def preprocess(image_path):
    # Convert all the images to size 299x299 as expected by the inception v3 model
    img = image.load_img(image_path, target_size=(299, 299))
    # Convert PIL image to numpy array of 3-dimensions
    x = image.img_to_array(img)
    # Add one more dimension
    x = np.expand_dims(x, axis=0)
    # preprocess the images using preprocess_input() from inception module
    x = preprocess_input(x)
    return x


In [20]:
# Load the inception v3 model
model = InceptionV3(weights='imagenet')

# Create a new model, by removing the last layer (output layer) from the inception v3
model_new = Model(model.input, model.layers[-2].output)

#Observer le modèle : la dernière layer donne un output de 2048 dimensions
model_new.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 149, 149, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 149, 149, 32) 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 149, 149, 32) 0           batch_normalization_1[0][0]      
____________________________________________________________________________________________

In [21]:
# Function to encode a given image into a vector of size (2048, )
def encode(image):
    image = preprocess(image) # preprocess the image
    fea_vec = model_new.predict(image) # Get the encoding vector for the image
    fea_vec = np.reshape(fea_vec, fea_vec.shape[1]) # reshape from (1, 2048) to (2048, )
    return fea_vec

#sanity check
feature_image_11 = encode(memes_path+'quebecois11.jpeg')
feature_image_11.shape

(2048,)

# 3. Décodeur

### 3.1 Word embeddings

In [22]:
# on veut cb de dimensions dans le word embedding? 50, 100, 200 ou 300 
EMBEDDING_DIM = 300

In [23]:
# first, build index mapping words in the embeddings set
# to their embedding vector

print('Indexing word vectors : english.')
glove_path = 'data/glove/'
embeddings_index_en = {}
with open(glove_path+ 'glove.6B.300d.txt') as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, 'f', sep=' ')
        embeddings_index_en[word] = coefs

print('Found %s word vectors.' % len(embeddings_index_en))

Indexing word vectors : english.
Found 400000 word vectors.


In [24]:
# first, build index mapping words in the embeddings set
# to their embedding vector

print('Indexing word vectors : french.')
fasttext_path = 'data/fasttext_fr/'
embeddings_index_fr = {}
with open(fasttext_path + 'cc.fr.300.vec') as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, 'f', sep=' ')
        embeddings_index_fr[word] = coefs

print('Found %s word vectors.' % len(embeddings_index_fr))

Indexing word vectors : french.
Found 2000000 word vectors.


In [25]:
# prepare embedding matrix
num_words = min(MAX_NB_WORDS, num_words_k)      # prend le nombre de mots total dans notre dict, et ajoute +1 par convention keras
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))   # initialise la matrice d'embedding
none_count = num_words

for word, i in word_index.items():
    if i >= MAX_NB_WORDS:
        continue
    embedding_vector = embeddings_index_fr.get(word)  #cherche le mot en français
    if embedding_vector is None:
        embedding_vector = embeddings_index_en.get(word) #si il ne l'avait pas en français, le cherche en anglais

    if embedding_vector is not None: #si il a trouvé un vecteur en francais ou en anglais, le mettre dans la matrice
        # sinon, ce sera encore des 0
        embedding_matrix[i] = embedding_vector
        none_count += -1

# load pre-trained word embeddings into an Embedding layer
# note that we set trainable = False so as to keep the embeddings fixed
embedding_layer = Embedding(num_words,
                            EMBEDDING_DIM,
                            embeddings_initializer=Constant(embedding_matrix),
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=True)  # on met ça pour que les embeddings puissent changer selon notre dataset

print('Il y a ' + str(none_count) + ' mots sans embeddings, sur un total de ' +str(num_words) + ' mots' )

Il y a 1370 mots sans embeddings, sur un total de 6000 mots


In [23]:
#sanity check : regarder le vecteur d'embedding pour le mot 'jamais'

print(index_word[100])
embedding_matrix[100]

jamais


array([-0.004     ,  0.0374    , -0.0041    ,  0.0242    , -0.0949    ,
       -0.0358    ,  0.0979    , -0.0073    , -0.0321    , -0.0261    ,
        0.0804    ,  0.0313    ,  0.0188    ,  0.0166    ,  0.0052    ,
        0.0432    , -0.0116    , -0.0063    , -0.0199    ,  0.0121    ,
        0.0415    ,  0.0021    , -0.0215    ,  0.0306    ,  0.0109    ,
        0.0347    , -0.1173    , -0.0637    ,  0.0064    ,  0.0249    ,
       -0.0024    ,  0.0264    , -0.0246    ,  0.0774    ,  0.0203    ,
        0.0208    , -0.0134    ,  0.0315    , -0.0124    , -0.0317    ,
        0.0255    , -0.0004    ,  0.0247    , -0.0103    , -0.0835    ,
       -0.0163    , -0.0328    ,  0.0029    ,  0.0062    , -0.0255    ,
       -0.0186    ,  0.0141    ,  0.0493    ,  0.022     , -0.0241    ,
        0.1245    , -0.0123    ,  0.027     , -0.0504    ,  0.0792    ,
        0.018     , -0.0071    ,  0.0008    ,  0.007     ,  0.0406    ,
        0.0092    , -0.0541    , -0.0224    ,  0.0137    ,  0.01

## 3.2 Créer les X et les Y -- ! Prend beaucoup de RAM !

In [83]:
#observer le format de nos captions : liste d'index
sequences[10]

[2,
 8,
 9,
 25,
 180,
 66,
 34,
 459,
 12,
 698,
 34,
 3027,
 16,
 52,
 880,
 9,
 2000,
 380,
 98,
 1195,
 107,
 13,
 12,
 699,
 698,
 3]

In [84]:
#pour chaque caption, prendre chaque paire X = 1er mot Y = 2e mot, X = 1er et 2e mot Y = 3e mot, etc 
X, y, X_img = list(), list(), list()

i2 = 0
for seq in sequences:
  for i in range(1, len(seq)):
    # split into input and output pair
    in_seq, out_seq = seq[:i], seq[i]
    # pad input sequence
    in_seq = pad_sequences([in_seq], maxlen=MAX_SEQUENCE_LENGTH)[0]
    # encode output sequence
    out_seq = to_categorical([out_seq], num_classes=num_words)[0]    
    # store
    X.append(in_seq)
    y.append(out_seq)
    X_img.append(img_list[i2])    #déterminer l'image associée

  i2 += 1

X = np.asarray(X)
y = np.asarray(y)

MemoryError: Unable to allocate 1.18 GiB for an array with shape (52909, 6000) and data type float32

In [85]:
#sanity check : X_img devrait contenir plusieurs fois la même image puisqu'elle est liée à chaque séquence
X_img

['conneriesqc0001.jpeg',
 'conneriesqc0001.jpeg',
 'conneriesqc0001.jpeg',
 'conneriesqc0001.jpeg',
 'conneriesqc0001.jpeg',
 'conneriesqc0001.jpeg',
 'conneriesqc0001.jpeg',
 'conneriesqc0001.jpeg',
 'conneriesqc0001.jpeg',
 'conneriesqc0001.jpeg',
 'conneriesqc0001.jpeg',
 'conneriesqc0002.jpeg',
 'conneriesqc0002.jpeg',
 'conneriesqc0002.jpeg',
 'conneriesqc0002.jpeg',
 'conneriesqc0002.jpeg',
 'conneriesqc0002.jpeg',
 'conneriesqc0002.jpeg',
 'conneriesqc0002.jpeg',
 'conneriesqc0002.jpeg',
 'conneriesqc0002.jpeg',
 'conneriesqc0002.jpeg',
 'conneriesqc0002.jpeg',
 'conneriesqc0002.jpeg',
 'conneriesqc0004.jpeg',
 'conneriesqc0004.jpeg',
 'conneriesqc0004.jpeg',
 'conneriesqc0004.jpeg',
 'conneriesqc0004.jpeg',
 'conneriesqc0004.jpeg',
 'conneriesqc0004.jpeg',
 'conneriesqc0004.jpeg',
 'conneriesqc0004.jpeg',
 'conneriesqc0004.jpeg',
 'conneriesqc0004.jpeg',
 'conneriesqc0004.jpeg',
 'conneriesqc0004.jpeg',
 'conneriesqc0004.jpeg',
 'conneriesqc0004.jpeg',
 'conneriesqc0004.jpeg',


In [86]:
#sanity check : le dernier élément de X[3] devrait être la position du 1 dans y[2]
np.set_printoptions(threshold=sys.maxsize)

print(X[2])
print(y[1])
print(X_img[2])

[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    2 1993   15]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 

In [87]:
#sanity check
print(X.shape)
print('Correspond au max length ' + str(MAX_SEQUENCE_LENGTH))
print(y.shape)
print('Correspond au min entre nbre de mots dans le vocab et MAX_NB_WORDS ' + str(num_words))
print(len(X_img))

(52909, 105)
Correspond au max length 105


AttributeError: 'list' object has no attribute 'shape'

## 3.3 Image embeddings

#### Créer le dictionnaire des images embedded

In [0]:
import time
start = time.time()

i = 0 
#pour toutes les images dans la liste, les encoder une fois et le mettre dans un dict. key = nom de l'Image, value = vecteur encodé
img_embed_dict = {}
for img in img_list:
  img_embed_dict[img] = encode(drive_path + memes_path + img)
  i += 1
  if i%50 == 0:
    print('Progrès : '+ str(i) + ' sur un total de ' + str(len(img_list)))

#pour toutes les images dans X_img (donc avec bcp de répétitions), aller chercher la value dans le dict
X_img_encoded = [img_embed_dict[img] for img in X_img]
X_img_encoded = np.asarray(X_img_encoded)


end = time.time()
print(end - start)

Progrès : 50 sur un total de 3820
Progrès : 100 sur un total de 3820
Progrès : 150 sur un total de 3820
Progrès : 200 sur un total de 3820
Progrès : 250 sur un total de 3820
Progrès : 300 sur un total de 3820
Progrès : 350 sur un total de 3820
Progrès : 400 sur un total de 3820
Progrès : 450 sur un total de 3820
Progrès : 500 sur un total de 3820
Progrès : 550 sur un total de 3820
Progrès : 600 sur un total de 3820
Progrès : 650 sur un total de 3820
Progrès : 700 sur un total de 3820
Progrès : 750 sur un total de 3820
Progrès : 800 sur un total de 3820
Progrès : 850 sur un total de 3820
Progrès : 900 sur un total de 3820
Progrès : 950 sur un total de 3820
Progrès : 1000 sur un total de 3820
Progrès : 1050 sur un total de 3820
Progrès : 1100 sur un total de 3820
Progrès : 1150 sur un total de 3820
Progrès : 1200 sur un total de 3820
Progrès : 1250 sur un total de 3820
Progrès : 1300 sur un total de 3820
Progrès : 1350 sur un total de 3820
Progrès : 1400 sur un total de 3820
Progrès : 14

In [0]:
### sauver le dictionnaire
dict_path = 'data/dict/'
now = datetime.now()
time_str = str(now)[:16]

np.save(dict_path + time_str + 'img_embed_dict.npy', img_embed_dict) 


#### Loader le dictionnaire des images embedded

In [25]:
# Load
dict_path = 'data/dict/'

img_embed_dict = np.load(dict_path + '2019-11-27 03_51img_embed_dict.npy',allow_pickle='TRUE').item()


In [0]:
#pour toutes les images dans X_img (donc avec bcp de répétitions), aller chercher la value dans le dict
X_img_encoded = [img_embed_dict[img] for img in X_img]
X_img_encoded = np.asarray(X_img_encoded)

## 3.4 Modèles

### Modèle 1 : LSTM sur texte, add les 2

In [26]:
model_name = 'LSTM_initial'

inputs1 = Input(shape=(2048,))
fe1 = Dropout(0.5)(inputs1)
fe2 = Dense(256, activation='relu')(fe1)
inputs2 = Input(shape=(MAX_SEQUENCE_LENGTH,))
se1 = embedding_layer(inputs2)
se2 = Dropout(0.5)(se1)
se3 = LSTM(256)(se2)
decoder1 = add([fe2, se3])
decoder2 = Dense(256, activation='relu')(decoder1)
outputs = Dense(num_words, activation='softmax')(decoder2)
model = Model(inputs=[inputs1, inputs2], outputs=outputs)

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])


### Modèle 2 : LSTM sur texte + sur combinaison des 2

In [37]:
model_name = 'LSTM_sur_both'

inputs1 = Input(shape=(2048,))
im1 = Dense(256, activation='relu')(inputs1)
im2 = RepeatVector(MAX_SEQUENCE_LENGTH)(im1)

inputs2 = Input(shape=(MAX_SEQUENCE_LENGTH,))
lm1 = embedding_layer(inputs2)
lm2 = Bidirectional(LSTM(256, return_sequences=True))(lm1)
lm3 = Dropout(0.5)(lm2)
lm4 = BatchNormalization()(lm3)
lm5 = TimeDistributed(Dense(EMBEDDING_DIM))(lm4)

decoder1 = concatenate([im2,lm5])
decoder2 = Dropout(0.5)(decoder1)
decoder3 = BatchNormalization()(decoder2)
decoder4 = Bidirectional(LSTM(1000,return_sequences=False))(decoder3)

outputs = Dense(num_words, activation='softmax')(decoder4)

model = Model(inputs=[inputs1, inputs2], outputs=outputs)

#ajouter une metric : perplexité
def perplexity(y_true, y_pred):
    cross_entropy = K.backend.mean(K.backend.categorical_crossentropy(y_true, y_pred))
    perplexity = K.backend.exp(cross_entropy)
    return perplexity

model.compile(loss="categorical_crossentropy", optimizer="SGD", metrics=[perplexity])

In [34]:
plot_model(model, to_file='temp.png')

OSError: `pydot` failed to call GraphViz.Please install GraphViz (https://www.graphviz.org/) and ensure that its executables are in the $PATH.

In [35]:
model.summary()


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 105)          0                                            
__________________________________________________________________________________________________
input_1 (InputLayer)            (None, 2048)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 105, 300)     1800000     input_2[0][0]                    
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 2048)         0           input_1[0][0]                    
____________________________________________________________________________________________

### Entraîner le modèle

In [39]:
### 15 EPOCHS
start = time()

model.fit([X_img_encoded, X], y, nb_epoch=15, batch_size=256) 

end = time()
print(end - start)


# sauver directement le modèle
now = datetime.now()
time_str = str(now)[:16]

model_path = 'data/model/'
model.save(model_path + model_name + time_str + '.h5')

NameError: name 'X_img_encoded' is not defined

In [ ]:
# Plot training & validation loss values
plt.plot(history.history['perplexity'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [0]:
#Examiner la embedding_layer
embed_weights = embedding_layer.get_weights()
print(index_word[7225])
print(embedding_matrix[7225])
print(embed_weights[0][7225])

In [0]:
# par comparaison, voici la matrice avant training d'un autre mot
print(index_word[10])
embedding_matrix[10]

### Loader le modèle

In [41]:
model_3 = load_model('data/model/LSTM_v2_3epochs.h5')
model_15 = load_model('data/model/LSTM_v2_15epochs.h5')
#model_50 = load_model('data/model/LSTM_v2_50epochs.h5')

/home/jeremi/anaconda3/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/home/jeremi/anaconda3/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


# 4. Prédictions

## Prédire sur une nouvelle image (sans beam search, mais probabiliste)

In [4]:
#créer un array avec une image test
img_path = 'data/test_images/'
count = 0
gen_path = 'data/GENERATED/'
liste_images = ['vince.jpg']

In [31]:
#fonction pour générer les images
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds) ### pcq la somme des preds doit être 1
    probas = np.random.multinomial(1, preds.T, 1)
    return np.argmax(probas)

def crop_center(pil_img, crop_width, crop_height):
    img_width, img_height = pil_img.size
    return pil_img.crop(((img_width - crop_width) // 2,
                        (img_height - crop_height) // 2,
                        (img_width + crop_width) // 2,
                        (img_height + crop_height) // 2))

def crop_max_square(pil_img):
    return crop_center(pil_img, min(pil_img.size), min(pil_img.size))


In [42]:
for choix in liste_images:
    img_test_path = img_path + choix


    #pour avoir une image dans les 3 modèles  
    name = 0
    for model in [model_3, model_15]:#, model_50]:
        name += 1
        if name == 1:
            model_name = 'model_3'
        if name == 2:
            model_name = 'model_15'
        if name == 3:
            model_name = 'model_50'


    #pour avoir une image avec plusieurs températures
    for temperature in [0.1, 0.2, 0.3, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 0.99]:
        ### VERSION RANDOM MULTINOMIAL : Choisit de façon probabiliste
        #créer un array pour le feeder dans le predict
        test = np.zeros((1,MAX_SEQUENCE_LENGTH))
        test[0][MAX_SEQUENCE_LENGTH-1] = start_tok

        #créer un array avec une image test
        img_test = encode(img_test_path)
        img_test = np.asarray(img_test).reshape(1,2048)

        while test[0][MAX_SEQUENCE_LENGTH-1] != end_tok and test[0][0] == 0 : #jusqu'à temps que le dernier index soit 2 (pcq 2 = eeee)
            answ = model.predict([img_test, test])   #prendre le vecteur avec le ssss et prédire le prochain mot
            id_pred = sample(answ, temperature= temperature)    #ça donne une réponse de dimension 1570. On prend l'index du mot le plus probable

            id_pred = str(id_pred)

            test = np.roll(test, -1)     #on bouge de 1 vers la gauche et on ajoute le id_pred
            test[0][MAX_SEQUENCE_LENGTH-1] = id_pred


        mots = []
        for value in test[0]:
            if value > 3:
                mots.append(index_word[value])
            output = ' '.join(mots)
            #print(output)

            #tokenizer.sequences_to_texts(list(test))

            #############################################

        #      try:
        img = Image.open(img_test_path)

        # Obtenir le plus grand carré possible
        img = crop_max_square(img)
        font_size = int(img.height/10)

        # Ajouter un rectangle blanc
        draw = ImageDraw.Draw(img)

        font = ImageFont.truetype("HelveticaNeueLt.ttf", font_size)
        text = output

        # Taille du texte
        w, h = draw.textsize(text, font)
        print("width font: {}".format(w))
        print("height font: {}".format(h))
        print("width image: {}".format(img.width))

        # Compter nombre de lignes
        if w > img.width:
            lineCount = math.ceil(w / img.width)
        else:
            lineCount = 1
        print("lineCount: {}".format(lineCount))

        # Créer un rectangle en fonction du nombre de lignes
        height_rect = ((10*2) + (lineCount * h))
        img = img.crop((0, -(height_rect), img.width, img.height))
        draw = ImageDraw.Draw(img)
        draw.rectangle((0, 0, img.width, height_rect), fill = 'white')

        # Déterminer la largeur d'une lettre
        letter_width, letter_height = draw.textsize('a', font = font)

        # Créer une liste de lignes
        lines = []

        if lineCount > 1:
            lastCut = 0
            isLast = False
            for i in range(0,lineCount):
                if lastCut == 0:
                    cut = (math.floor(img.width/letter_width)) * i
                    #cut = (len(text) / lineCount) * i
                else:
                    cut = lastCut
                if i < lineCount-1:
                    nextCut = (math.floor(img.width/letter_width)) * (i+1)
                    # nextCut = (len(text) / lineCount) * i
                else:
                    nextCut = len(text)
                    isLast = True
                print("cut: {} -> {}".format(cut, nextCut))

                # Ne pas couper des mots en deux
                if nextCut == len(text) or text[int(nextCut)] == " ":
                    print("may cut")
                else:
                    print("may not cut")
                    while text[int(nextCut)] != " ":
                        nextCut += 1
                    print("new cut: {}".format(nextCut))

                line = text[int(cut):int(nextCut)].strip()

                # Vérifier si la ligne fitte
                w, h = draw.textsize(line, font)
                if not isLast and w > img.width:
                    print("overshot")
                    nextCut -= 1
                    while text[nextCut] != " ":
                        nextCut -= 1
                    print("new cut: {}".format(nextCut))

                lastCut = nextCut
                lines.append(text[int(cut):int(nextCut)].strip())

        else:
            lines.append(text)
        #print(lines)

        for i in range(0, lineCount):
            w, h = draw.textsize(lines[i], font)
            draw.text((10, (10 + i*h)), lines[i], (0,0,0), font=font)

        img.save(gen_path  +choix + '_' + model_name + '_' + str(temperature) + ' Probabiliste.jpg')
        count +=1
    #except: pass

width font: 5256
height font: 132
width image: 1125
lineCount: 5
cut: 0 -> 19
may not cut
new cut: 23
overshot
new cut: 17
cut: 17 -> 38
may cut
cut: 38 -> 57
may not cut
new cut: 58
cut: 58 -> 76
may not cut
new cut: 79
cut: 79 -> 113
may cut
width font: 2291
height font: 131
width image: 1125
lineCount: 3
cut: 0 -> 19
may cut
cut: 19 -> 38
may not cut
new cut: 40
cut: 40 -> 48
may cut
width font: 5256
height font: 132
width image: 1125
lineCount: 5
cut: 0 -> 19
may not cut
new cut: 23
overshot
new cut: 17
cut: 17 -> 38
may cut
cut: 38 -> 57
may not cut
new cut: 58
cut: 58 -> 76
may not cut
new cut: 79
cut: 79 -> 113
may cut
width font: 5256
height font: 132
width image: 1125
lineCount: 5
cut: 0 -> 19
may not cut
new cut: 23
overshot
new cut: 17
cut: 17 -> 38
may cut
cut: 38 -> 57
may not cut
new cut: 58
cut: 58 -> 76
may not cut
new cut: 79
cut: 79 -> 113
may cut
width font: 5037
height font: 132
width image: 1125
lineCount: 5
cut: 0 -> 19
may not cut
new cut: 23
overshot
new cut: 17

## Prédire sur une nouvelle image : BEAM SEARCH

In [0]:
def preds_to_list_chosen(preds,temperature=1.0):
    
    list_chosen = []
    # helper function to sample an index from a probability array
    preds_arr = np.asarray(preds).astype('float64')
    preds_log = np.log(preds_arr) / temperature
    exp_preds = np.exp(preds_log)
    preds_sum = exp_preds / np.sum(exp_preds) ### pcq la somme des preds doit être 1
    
    preds_count = 0 
    while len(list_chosen) < 1:
      preds_count += 1
      #print('Essai #' + str(preds_count))
      probas = np.random.multinomial(k, preds_sum.T, 1)

      list_chosen = np.argwhere(probas == np.amax(probas)).flatten().tolist()
      list_chosen = list(filter(lambda a: a != 0, list_chosen))

      #print('Nbre de mots considérés par le beam search : ' +str(len(list_chosen)))

    return list_chosen


def dict_prob_to_top_vectors(dict_prob):
    keys_list = list(dict_prob.keys())
    keys_list.sort(reverse=True)

    top_k_vectors = []
    top_1 = dict_prob[keys_list[0]]

    for i in range(k):
        try:
          vect = dict_prob[keys_list[i]]
          top_k_vectors.append(vect)
        except: pass

    return top_1, top_k_vectors


In [0]:
img_path = 'data/test_images/'
count = 0
gen_path = 'data/GENERATED/'
printed_count = 0
error_count = 0
k=3

In [0]:
### LOOP :
for choix in ['test_vince.jpg', 'test_blanc.jpg', 'test_sapin.jpg', 'test_meme.jpg', 'test_justine.jpg']:
  img_test_path = drive_path + img_path + choix

  name = 0

  for model in [model_3, model_15, model_50]:
    name += 1
    if name == 1:
      model_name = 'model_3'
    if name == 2:
      model_name = 'model_15'
    if name == 3:
      model_name = 'model_50'


    for temperature in [0.1, 0.2, 0.3, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 0.99]:

      dict_prob = {}

      test = np.zeros((1,MAX_SEQUENCE_LENGTH))
      test[0][MAX_SEQUENCE_LENGTH-1] = start_tok
      test_shape = test.shape

      img_test = encode(img_test_path)
      img_test = np.asarray(img_test).reshape(1,2048)

      ### premier loop
      preds = model.predict([img_test,test])
      list_chosen = preds_to_list_chosen(preds, temperature = temperature)

      for id_pred in list_chosen:
          test_2 = test

          test_2 = np.roll(test_2, -1)     #on bouge de 1 vers la gauche et on ajoute le id_pred
          test_2[0][MAX_SEQUENCE_LENGTH-1] = id_pred


          prob = model.predict([img_test, test])[0][id_pred] # on veut connaître p(x1,x2). on calcule donc p(x2|x1) en prenant l'ancien vecteur et en sélectionnant l'index qui correspond au id_pred
          dict_prob[prob] = test_2


      top_1 , top_k_vectors = dict_prob_to_top_vectors(dict_prob)

      count = 0
      ### loops suivants
      while top_1[0][MAX_SEQUENCE_LENGTH-1] != end_tok and top_1[0][0] == 0 : #jusqu'à temps que le dernier index soit 2 (pcq 2 = eeee)
        dict_prob = {}
        count += 1
        #print('Loop ' +str(count))

        for vect in top_k_vectors:
          vect = vect.reshape(test_shape)

          preds = model.predict([img_test,vect])
          list_chosen = preds_to_list_chosen(preds, temperature = temperature)

          past_vect = vect

          for id_pred in list_chosen:
            #print(id_pred)
            test_2 = past_vect

            test_2 = np.roll(test_2, -1)     #on bouge de 1 vers la gauche et on ajoute le id_pred
            test_2[0][MAX_SEQUENCE_LENGTH-1] = id_pred

            ###
            prob=1
            for i in range(1,count+1):
              prev_target = test_2[0][MAX_SEQUENCE_LENGTH-i].astype(int) #la cible : le dernier chiffre qui va apparaître après que le vecteur soit roulé
              prev_vect = np.roll(test_2, i) #on bouge de i vers la droite
              prev_vect[0][:i] = 0          #on remplace les i premiers chiffre (roulés) par 0
              curr_prob =  model.predict([img_test,prev_vect])[0][prev_target]
              prob = prob*curr_prob

            #prob = model.predict([img_test, past_vect])[0][id_pred] # on veut connaître p(x1,x2). on calcule donc p(x2|x1) en prenant l'ancien vecteur et en sélectionnant l'index qui correspond au id_pred
            ###

            dict_prob[prob] = test_2

            #print(dict_prob)

        top_1 , top_k_vectors = dict_prob_to_top_vectors(dict_prob)

      #tokenizer.sequences_to_texts(top_1)
      mots = []
      for value in top_1[0]:
        if value > 3:
          mots.append(index_word[value])
      output = ' '.join(mots)

      #############
      try:
        from PIL import Image, ImageDraw, ImageFont
        import math

        img = Image.open(img_test_path)

        # Obtenir le plus grand carré possible
        def crop_center(pil_img, crop_width, crop_height):
            img_width, img_height = pil_img.size
            return pil_img.crop(((img_width - crop_width) // 2,
                                (img_height - crop_height) // 2,
                                (img_width + crop_width) // 2,
                                (img_height + crop_height) // 2))

        def crop_max_square(pil_img):
            return crop_center(pil_img, min(pil_img.size), min(pil_img.size))
        img = crop_max_square(img)
        font_size = int(img.height/10)

        # Ajouter un rectangle blanc
        draw = ImageDraw.Draw(img)

        font = ImageFont.truetype("HelveticaNeueLt.ttf", font_size)
        text = output

        # Taille du texte
        w, h = draw.textsize(text, font)
        print("width font: {}".format(w))
        print("height font: {}".format(h))
        print("width image: {}".format(img.width))

        # Compter nombre de lignes
        if w > img.width:
            lineCount = math.ceil(w / img.width)
        else:
            lineCount = 1
        print("lineCount: {}".format(lineCount))

        # Créer un rectangle en fonction du nombre de lignes
        height_rect = ((10*2) + (lineCount * h))
        img = img.crop((0, -(height_rect), img.width, img.height))
        draw = ImageDraw.Draw(img)
        draw.rectangle((0, 0, img.width, height_rect), fill = 'white')

        # Déterminer la largeur d'une lettre
        letter_width, letter_height = draw.textsize('a', font = font)

        # Créer une liste de lignes
        lines = []

        if lineCount > 1:
            lastCut = 0
            isLast = False
            for i in range(0,lineCount):
                if lastCut == 0:
                    cut = (math.floor(img.width/letter_width)) * i
                    #cut = (len(text) / lineCount) * i
                else:
                    cut = lastCut
                if i < lineCount-1:
                    nextCut = (math.floor(img.width/letter_width)) * (i+1)
                    # nextCut = (len(text) / lineCount) * i
                else:
                    nextCut = len(text)
                    isLast = True
                print("cut: {} -> {}".format(cut, nextCut))

                # Ne pas couper des mots en deux
                if nextCut == len(text) or text[int(nextCut)] == " ":
                    print("may cut")
                else:
                    print("may not cut")
                    while text[int(nextCut)] != " ":
                        nextCut += 1
                    print("new cut: {}".format(nextCut))

                line = text[int(cut):int(nextCut)].strip()

                # Vérifier si la ligne fitte
                w, h = draw.textsize(line, font)
                if not isLast and w > img.width:
                    print("overshot")
                    nextCut -= 1
                    while text[nextCut] != " ":
                        nextCut -= 1
                    print("new cut: {}".format(nextCut))

                lastCut = nextCut
                lines.append(text[int(cut):int(nextCut)].strip())

        else:
            lines.append(text)
        #print(lines)

        for i in range(0, lineCount):
            w, h = draw.textsize(lines[i], font)
            draw.text((10, (10 + i*h)), lines[i], (0,0,0), font=font)

        img.save(drive_path + gen_path + choix + '_' + model_name + '_' + str(temperature) + ' BeamSearch.jpg')
        count +=1

        printed_count += 1
        print('Imprimés : ' + str(printed_count))
      except: 
        error_count += 1
        print('Erreurs : ' + str(error_count))

width font: 3302
height font: 351
width image: 3024
lineCount: 2
cut: 0 -> 19
may not cut
Erreurs : 1
width font: 5037
height font: 351
width image: 3024
lineCount: 2
cut: 0 -> 19
may not cut
new cut: 25
overshot
new cut: 18
cut: 18 -> 39
may cut
Imprimés : 1
width font: 3374
height font: 351
width image: 3024
lineCount: 2
cut: 0 -> 19
may cut
cut: 19 -> 26
may cut
Imprimés : 2
width font: 3302
height font: 351
width image: 3024
lineCount: 2
cut: 0 -> 19
may not cut
Erreurs : 2
width font: 5792
height font: 351
width image: 3024
lineCount: 2
cut: 0 -> 19
may not cut
new cut: 20
cut: 20 -> 44
may cut
Imprimés : 3
width font: 2692
height font: 351
width image: 3024
lineCount: 1
Imprimés : 4
width font: 5779
height font: 355
width image: 3024
lineCount: 2
cut: 0 -> 19
may not cut
new cut: 21
cut: 21 -> 46
may cut
Imprimés : 5
width font: 5082
height font: 351
width image: 3024
lineCount: 2
cut: 0 -> 19
may not cut
new cut: 24
cut: 24 -> 41
may cut
Imprimés : 6
width font: 5061
height font